In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import json
import pandas as pd

In [2]:
options =uc.ChromeOptions()
options.add_argument('--disable-popup-blocking')
driver = uc.Chrome(options)
driver.maximize_window()

In [3]:
driver.get('https://www.gelbeseiten.de/suche/steuerberatung/essen?umkreis=50000')
time.sleep(3)

In [84]:
driver.find_element(By.XPATH, '//span[@class="cmpboxbtnspan"]/a').click()

In [85]:
total = 100
loadmorebutton = driver.find_element(By.XPATH, '//a[@id="mod-LoadMore--button"]')
loaded = []

while len(driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article'))!= total:
    try:
        driver.execute_script("arguments[0].scrollIntoView(false);", loadmorebutton)
        loadmorebutton.click()
        loaded.append(len(driver.find_elements(By.XPATH, '//div[@id="teilnehmer_block"]/div/article')))
        time.sleep(3)
    except:        
        time.sleep(3)
print(loaded)


[50, 60, 70, 80, 90]


In [86]:
Names, TaxAdvices, Emails, Locations, Distances, Phones, Websites = [],[],[],[],[],[],[]

profiles = driver.find_elements(By.XPATH, '//*[@id="gs_treffer"]/article')

for profile in profiles:
    driver.execute_script("arguments[0].scrollIntoView();", profile)
    time.sleep(2)
    
    distance = profile.find_element(By.XPATH, './/span[@class="mod-AdresseKompakt__entfernung"]').text
    
    link = profile.find_element(By.XPATH, ".//a").get_attribute('href')
    page = driver.execute_script("window.open('{}', '_blank');".format(link))
    driver.switch_to.window(driver.window_handles[1])
    
    try:
        name = driver.find_element(By.XPATH, '//div[@class="mod-TeilnehmerKopf__Name-wrapper"]/h1').text
        Names.append(name)
    except:
        Names.append(None)

    '''try:
        taxadv =driver.find_element(By.XPATH, '//div[@class="mod-TeilnehmerKopf__branchen"]/span[3]').text.replace('TAX ADVICE: ', '')
        TaxAdvices.append(taxadv)
    except:
         TaxAdvices.append(None)'''

    try:
        email = driver.find_element(By.XPATH, '//div[@id="email_versenden"]').get_attribute("data-link").removeprefix('mailto:').removesuffix("?subject=Anfrage über Gelbe Seiten")
        Emails.append(email)
    except:
        Emails.append(None)
    
    try:
        location = ((driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span').text) + ' ' +
                    (driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span[2]').text) + ' ' +
                    (driver.find_element(By.XPATH, '//address[@class="mod-TeilnehmerKopf__adresse"]/div/span[3]').text))
        Locations.append(location)
    except:
        Locations.append(None)

    try:
        Distances.append(distance)
    except:
        Distances.append(None)

    try:
        phone = driver.find_element(By.XPATH, '//span[@data-role="telefonnummer"]/a/span').text
        Phones.append(phone)
    except:
        Phones.append(None)

    try:
        website = driver.find_element(By.XPATH, '//a[@data-wipe-realview="detailseite_webadresse"]').get_attribute("href")
        Websites.append(website)
    except:
        Websites.append(None)
    
    print(f'{name}":" {taxadv}":"{email}":"{location}":"{distance}":"{phone}":"{website}":" ')

    time.sleep(2)
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)

data = pd.DataFrame(list(zip(Names, TaxAdvices, Emails, Locations, Distances, Phones, Websites)),
                    columns= ["Name", "TaxAdvice", "Email", "Location", "Distance", "Phone", "Website"])
data.to_csv('german.csv', index=False, encoding='utf-8-sig')
driver.quit()


Roland Franz & Partner Steuerberater":" STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER":"kontakt@franz-partner.de":"Moltkeplatz 1, 45138 Essen":"1,3 km":"0201 8 10 95-0":"http://www.franz-partner.de/":" 
Tennagels Angelika":" STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER":"info@stb-tennagels.de":"Altenessener Str. 208, 45326 Essen":"3,3 km":"0201 31 97 00 41":"https://www.stb-tennagels.de/":" 
Anwalts-/ Steuerbüro Küppers-Tonner G.":" STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER":"steuerberatung@kueppers-tonner.de":"Gerichtsstr. 37, 45355 Essen-Borbeck-Mitte":"4,6 km":"0201 6 13 58-0":"http://www.kueppers-tonner.de/":" 
Kremer Helmut":" STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER":"info@helmut-kremer.de":"Solinger Str. 12, 45481 Mülheim an der Ruhr-Saarn/Mintard":"11,8 km":"0208 4 84 96-0":"http://www.helmut-kremer.de/":" 
D.DO Dautzenberg Ochsenschläger Steuerberater Patnerschaft mbB":" STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER":"mail@ddo-partner.de":"Neumühler Str. 164, 47167 Duisbu

In [91]:
pd.set_option('display.max_rows', 500)
germandata = data.drop('TaxAdvice', axis=1)
germandata

,Name,Email,Location,Distance,Phone,Website
0,Roland Franz & Partner Steuerberater,kontakt@franz-partner.de,"Moltkeplatz 1, 45138 Essen","1,3 km",0201 8 10 95-0,http://www.franz-partner.de/
1,Tennagels Angelika,info@stb-tennagels.de,"Altenessener Str. 208, 45326 Essen","3,3 km",0201 31 97 00 41,https://www.stb-tennagels.de/
2,Anwalts-/ Steuerbüro Küppers-Tonner G.,steuerberatung@kueppers-tonner.de,"Gerichtsstr. 37, 45355 Essen-Borbeck-Mitte","4,6 km",0201 6 13 58-0,http://www.kueppers-tonner.de/
3,Kremer Helmut,info@helmut-kremer.de,"Solinger Str. 12, 45481 Mülheim an der Ruhr-Sa...","11,8 km",0208 4 84 96-0,http://www.helmut-kremer.de/
4,D.DO Dautzenberg Ochsenschläger Steuerberater ...,mail@ddo-partner.de,"Neumühler Str. 164, 47167 Duisburg-Neumühl","15,4 km",0203 9 95 30-0,https://www.ddo-partner.de/
5,Bölting + Kollegen,steuerberatung@boelting.com,"Auf der Reihe 2, 45884 Gelsenkirchen-Rotthausen","6,6 km",0209 9 41 44-0,http://www.steuerweg.com/
6,Dünkel & Köster Partnerschaft mbB Steuerberater,info@steuerberatung-dk.de,"Sandstr. 9, 46117 Oberhausen-Eisenheim","11,8 km",0208 6 99 69-0,https://www.steuerberatung-dk.de/
7,Michael te Heesen Steuerberater,info@te-heesen.de,"Augustin-Wibbelt-Str. 4, 46242 Bottrop-Fuhlenb...","10,6 km",02041 5 29 59,https://www.te-heesen.de/
8,Kremer Helmut,info@helmut-kremer.de,"Solinger Str. 12, 45481 Mülheim an der Ruhr-Sa...","11,8 km",0208 4 84 96-0,http://www.helmut-kremer.de/
9,Henscheid Klaus Dipl.-Finanzwirt,info@klaushenscheid.de,"Heisinger Str. 496, 45259 Essen-Heisingen",7 km,0201 8 46 03-0,http://www.klaushenscheid.de/


In [87]:
data

,Name,TaxAdvice,Email,Location,Distance,Phone,Website
0,Roland Franz & Partner Steuerberater,STEUERBERATUNG: SCHWERPUNKT ERBSCHAFTSSTEUER,kontakt@franz-partner.de,"Moltkeplatz 1, 45138 Essen","1,3 km",0201 8 10 95-0,http://www.franz-partner.de/
1,Tennagels Angelika,None,info@stb-tennagels.de,"Altenessener Str. 208, 45326 Essen","3,3 km",0201 31 97 00 41,https://www.stb-tennagels.de/
2,Anwalts-/ Steuerbüro Küppers-Tonner G.,None,steuerberatung@kueppers-tonner.de,"Gerichtsstr. 37, 45355 Essen-Borbeck-Mitte","4,6 km",0201 6 13 58-0,http://www.kueppers-tonner.de/
3,Kremer Helmut,None,info@helmut-kremer.de,"Solinger Str. 12, 45481 Mülheim an der Ruhr-Sa...","11,8 km",0208 4 84 96-0,http://www.helmut-kremer.de/
4,D.DO Dautzenberg Ochsenschläger Steuerberater ...,None,mail@ddo-partner.de,"Neumühler Str. 164, 47167 Duisburg-Neumühl","15,4 km",0203 9 95 30-0,https://www.ddo-partner.de/
...,...,...,...,...,...,...,...
95,ALEFF & Partner GmbH,None,info@aleff-partner.de,"Clemens-August-Str. 33, 46282 Dorsten-Hardt","22,4 km",02362 91 55-0,https://www.aleff-partner.de/
96,Bataille-Maas Dipl.-Ök.,None,info@steuerberatung-bataille.de,"Kohlenstr. 36, 44795 Bochum-Weitmar","12,9 km",0234 51 69 91-0,http://www.steuerberatung-bataille.de/
97,Ursula Schulte-Küch Steuerberaterin,None,steuerberatung@schulte-kuech.de,"Wilhelmstr. 15, 44649 Herne-Wanne","13,2 km",02325 7 52 63,http://www.ursula-schulte-kuech-steuerberateri...
98,Albers Steuerberatungsgesellschaft mbH & Co. KG,None,#,"Hardt 38, 40764 Langenfeld-Immigrath","37,7 km",02173 97 78 64,http://www.albers-steuerberater.de/
